In [2]:
from deepface import DeepFace


23-12-16 16:41:30 - Directory C:\Users\pedro/.deepface created
23-12-16 16:41:30 - Directory C:\Users\pedro/.deepface/weights created


In [47]:
result = DeepFace.find(img_path = "./tests/im2.jpg", db_path = "./db_rostos")

23-12-16 17:09:03 - ⚠️ Representations for images in ./db_rostos folder were previously stored in representations_vgg_face.pkl. If you added new instances after the creation, then please delete this file and call find function again. It will create it again.
23-12-16 17:09:03 - There are 2 representations found in representations_vgg_face.pkl
23-12-16 17:09:03 - find function lasts 0.34477734565734863 seconds


In [49]:
len(result[0])

0

É necessário excluir o arquivo representations_vgg_face.pkl quando adicionar uma pessoa na base de dados!

In [25]:
result = DeepFace.find(img_path = "./tests/im2.jpg", db_path = "./db_rostos")
result[0].identity[0].split('/')[2].split('.')[0]

'Pedro'

In [40]:
res = DeepFace.analyze('db_rostos\Eduardo.jpg',actions = ['age', 'gender', 'emotion'])

Action: emotion: 100%|██████████| 3/3 [00:00<00:00,  4.78it/s]


In [33]:
res = DeepFace.analyze('db_rostos\Eduardo.jpg')

Action: race: 100%|██████████| 4/4 [00:00<00:00,  4.23it/s]  


In [41]:
res

[{'age': 29,
  'region': {'x': 494, 'y': 75, 'w': 272, 'h': 272},
  'face_confidence': 6.765703093376942,
  'gender': {'Woman': 1.380640920251608, 'Man': 98.6193597316742},
  'dominant_gender': 'Man',
  'emotion': {'angry': 0.003379104964551516,
   'disgust': 9.746372597874142e-07,
   'fear': 0.05282876663841307,
   'happy': 99.9005138874054,
   'sad': 0.03083309275098145,
   'surprise': 0.0003870490672852611,
   'neutral': 0.01205714070238173},
  'dominant_emotion': 'happy'}]

In [9]:
res['dominant_emotion']

'happy'

In [94]:
from ultralytics import YOLO
model = YOLO('runs/detect/FacialDetection2/weights/best.pt')
# results = model.predict(source='0', show=True)
results = model.predict('lacto.jpg')
box = results[0].boxes.xyxy.tolist()


image 1/1 d:\GitHub\FaceRecognition_FeatureExtractor\lacto.jpg: 640x480 5 Faces, 10.2ms
Speed: 2.0ms preprocess, 10.2ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 480)


In [89]:
box

[[407.3908996582031, 60.7301025390625, 886.41162109375, 315.52703857421875]]

In [96]:
import cv2
image = cv2.imread("lacto.jpg") 
window_name = 'image'
for face in box:
    try:
        # Pontos XY
        x0 = int(face[0])
        y0 = int(face[1])
        x1 = int(face[2])
        y1 = int(face[3])

        # Recorte do rosto
        individual_face = image[y0:y1, x0:x1]

        # Inicio e Fim do rosto
        start_point = (x0, y0)
        end_point = (x1, y1)

        # Inicio e fim dos textos
        start_space = (x0, y1)
        end_space = (x1, y1+100)
        
        # Cor e espessura do retrato
        color = (172, 52, 52)
        thickness = 2

        # Dados sobre o rosto
        person = DeepFace.find(individual_face, db_path = "./db_rostos")
        per = 'N/A' if len(person[0]) == 0 else person[0].identity[0].split('/')[2].split('.')[0]
        res = DeepFace.analyze(individual_face)[0]

        # Retrato na foto original e espaco para o texto
        image = cv2.rectangle(image, start_point, end_point, color, thickness)
        image = cv2.rectangle(image, start_space, end_space, color, -1) 

        # # Resultados do rosto encontrado
        image = cv2.putText(image, f'Person: {per}', (x0+5, y1+18), cv2.FONT_HERSHEY_SIMPLEX,  
                    0.7, (255, 255, 255), 1) 
        image = cv2.putText(image, f'Gender: {res["dominant_gender"]}', (x0+5, y1+38), cv2.FONT_HERSHEY_SIMPLEX,  
                    0.7, (255, 255, 255), 1) 
        image = cv2.putText(image, f'Age: {res["age"]}', (x0+5, y1+58), cv2.FONT_HERSHEY_SIMPLEX,  
                    0.7, (255, 255, 255), 1) 
        image = cv2.putText(image, f'Emotion: {res["dominant_emotion"]}', (x0+5, y1+78), cv2.FONT_HERSHEY_SIMPLEX,  
                    0.7, (255, 255, 255), 1) 
        image = cv2.putText(image, f'Race: {res["dominant_race"]}', (x0+5, y1+98), cv2.FONT_HERSHEY_SIMPLEX,  
                    0.7, (255, 255, 255), 1) 
        
    except:
        pass
    
    # break
cv2.imshow(window_name, image)
cv2.waitKey(0) 
cv2.destroyAllWindows() 

23-12-16 17:27:09 - ⚠️ Representations for images in ./db_rostos folder were previously stored in representations_vgg_face.pkl. If you added new instances after the creation, then please delete this file and call find function again. It will create it again.
23-12-16 17:27:09 - There are 2 representations found in representations_vgg_face.pkl
23-12-16 17:27:09 - find function lasts 0.40056538581848145 seconds


Action: race: 100%|██████████| 4/4 [00:01<00:00,  3.64it/s]  


23-12-16 17:27:10 - ⚠️ Representations for images in ./db_rostos folder were previously stored in representations_vgg_face.pkl. If you added new instances after the creation, then please delete this file and call find function again. It will create it again.
23-12-16 17:27:10 - There are 2 representations found in representations_vgg_face.pkl
23-12-16 17:27:10 - find function lasts 0.3313314914703369 seconds


Action: race: 100%|██████████| 4/4 [00:00<00:00,  4.20it/s]  


23-12-16 17:27:11 - ⚠️ Representations for images in ./db_rostos folder were previously stored in representations_vgg_face.pkl. If you added new instances after the creation, then please delete this file and call find function again. It will create it again.
23-12-16 17:27:11 - There are 2 representations found in representations_vgg_face.pkl
23-12-16 17:27:12 - find function lasts 0.32947468757629395 seconds


Action: race: 100%|██████████| 4/4 [00:01<00:00,  3.96it/s]  


23-12-16 17:27:13 - ⚠️ Representations for images in ./db_rostos folder were previously stored in representations_vgg_face.pkl. If you added new instances after the creation, then please delete this file and call find function again. It will create it again.
23-12-16 17:27:13 - There are 2 representations found in representations_vgg_face.pkl
23-12-16 17:27:13 - ⚠️ Representations for images in ./db_rostos folder were previously stored in representations_vgg_face.pkl. If you added new instances after the creation, then please delete this file and call find function again. It will create it again.
23-12-16 17:27:13 - There are 2 representations found in representations_vgg_face.pkl
